# Set up Notebook

In [1]:
#!pip install --upgrade google-cloud-bigquery[pandas]

In [1]:
from google.cloud import bigquery

In [2]:
%reload_ext google.cloud.bigquery

In [3]:
client=bigquery.Client()

C:\Users\rasulit\AppData\Local\anaconda3\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


OSError: Project was not passed and could not be determined from the environment.

In [4]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [68]:
import os
import sys
from functools import reduce
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, String, MetaData, Integer
from sqlalchemy import inspect
import psycopg2
import random 
import string
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from itertools import accumulate
from itertools import repeat
import time
import xlsxwriter
import re
import jpype
import asposecells

In [69]:
rdr_where = '''

 withdrawal status = 0 and is_test_participant = 0
      p.hpo_id != 21 and p.is_ghost_id is null and
     (ps.email NOT LIKE '%@example.com' or ps.email IS NULL) and
     (REPLACE(REPLACE(REPLACE(REPLACE(ps.phone_number, '(', ''), ')', ''), '-', ''), ' ', '') not like '4442%' or ps.phone_number IS NULL) and
     (REPLACE(REPLACE(REPLACE(REPLACE(ps.login_phone_number, '(', ''), ')', ''), '-', ''), ' ', '')  not like '4442%' or ps.login_phone_number IS NULL)'''

# Credentials

In [70]:
# Insert your username into the textbox and hit enter
username = input("Please enter your username: ")

Please enter your username:  trasuli


In [71]:
try:
    from getpass import getpass
except ModuleNotFoundError:
    !pip install getpass4
    from getpass import getpass

password = getpass("Please enter your password: ")

Please enter your password:  ········


In [72]:
import mysql.connector

connect_options = {
   'user': username,
   'password':password, 
   'host':'localhost',
   'port':'9905',
   'database':'rdr',
   'raise_on_warnings': True}


## The next line of code is commented out because you do not need to run it 
# if you did it in the co8mmand line
# as shown in step 1

cnx = mysql.connector.connect(**connect_options)

# Match on participants on:

## Email AND phone 

In [73]:
######################################
print('Getting the data from the database...')
##########################431############

query = ('''  
    SELECT
       ps.first_name,ps.last_name,ps.phone_number,ps.email,date_of_birth,ps.participant_id,o.display_name,ps.organization_id,ps.hpo_id,ps.participant_origin,ps.consent_for_study_enrollment_authored,
       p.is_ghost_id, ps.zip_code,ps.state_id,city,ps.street_address,ps.ehr_status,ps.login_phone_number,
    CASE 
        WHEN
         (ps.hpo_id = 21 or p.is_ghost_id is not null) or
         (lower(ps.email) LIKE '%@example.com') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.login_phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%')
           THEN 'Yes'
           else "No" END as fake_test_participant_flag
    FROM participant_summary as ps
    left join organization as o on o.organization_id=ps.organization_id
    left outer join participant as p USING(participant_id)
    where (phone_number,email) in
                                    (SELECT
                                        phone_number,email
                                    FROM participant AS p
                                    Left outer Join participant_summary as ps
                                    on p.participant_id=ps.participant_id
                                    Group By 1,2
                                    having count(*)>1
                                    )
                                  and ps.withdrawal_status != 1
    order by 1,2,3
''')

#Now execute the query and put it it into a dataframe
rdr = pd.read_sql(query, cnx)

Getting the data from the database...


In [74]:
rdr.shape

(1161, 19)

In [75]:
rdr['email']=rdr['email'].str.lower()

In [76]:
rdr['last_name']=rdr['last_name'].str.lower()

In [77]:
rdr['first_name']=rdr['first_name'].str.lower()

In [78]:
rdr['first_name'] = rdr['first_name'].str.replace('é','e')
rdr['first_name'] = rdr['first_name'].str.replace('í','i')
rdr['last_name'] = rdr['last_name'].str.replace('é','e')
rdr['last_name'] = rdr['last_name'].str.replace('í','i')
rdr['last_name'] = rdr['last_name'].str.replace('á','a')
rdr['last_name'] = rdr['last_name'].str.replace('ó','o')

In [79]:
rdr['flag_1'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth','email','phone_number','ehr_status'],keep=False), 'Yes','No')

In [80]:
rdr['flag_2'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth'],keep=False), 'Yes','No')

In [81]:
rdr['flag_2_case_control'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth'],keep=False), 'Yes','No')

In [82]:
rdr['flag_3'] = np.where(rdr.duplicated(['email','phone_number'],keep=False), 'Yes','No')

In [83]:
rdr['flag_3_case_control'] = np.where(rdr.duplicated(['email','phone_number'],keep=False), 'Yes','No')

In [84]:
rdr.loc[rdr['flag_3_case_control'] == 'Yes', 'flag_2'] = 'No'

In [85]:
rdr.loc[rdr['flag_2_case_control'] == 'Yes', 'flag_3'] = 'No'

In [86]:
rdr.head(5)

,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,hpo_id,participant_origin,...,city,street_address,ehr_status,login_phone_number,fake_test_participant_flag,flag_1,flag_2,flag_2_case_control,flag_3,flag_3_case_control
0,"'""><asd{{7*7}}","'""><asd{{7*7}}",1231515151,ahmed_hamada@wearehackerone.com,1993-02-03,823463681,None,NaN,21,vibrent,...,"'""><asd","asd'""><asd",0,5087695036,Yes,No,No,No,No,No
1,{{7*7}},mitnick,3054456789,dondata@wearehackerone.com,1980-01-01,796072957,None,NaN,24,vibrent,...,miami,13701 sw 66th st,0,7863060580,No,No,No,No,No,No
2,{{7*7}},squirtle,9097643579,h1_squirtle+1@wearehackerone.com,1992-02-22,563775309,None,NaN,21,vibrent,...,CA,squirtle,0,None,Yes,No,No,No,No,No
3,{{programname}},{{programname}},4172253115,4bg0p@wearehackerone.com,1995-03-18,175304228,None,NaN,0,vibrent,...,{{programName}},{{programName}},0,None,No,No,No,No,Yes,Yes
4,"<img src=""//www.deviantart.com/adrianoramosofh...",ou,4172253115,4bg0p@wearehackerone.com,1995-03-18,230614263,None,NaN,0,vibrent,...,oui,oui,0,None,No,No,No,No,Yes,Yes


franklin	torres	

In [25]:
rdr.to_csv("..\\Downloads\\dups.csv")

In [87]:
rdr['participant_origin'].value_counts()

vibrent    1161
Name: participant_origin, dtype: int64

In [88]:
dublicated_email_phone=rdr[["first_name","last_name","phone_number","email","date_of_birth"]]

In [89]:
dublicated_email_phone["dublicated_email_phone"]=1

In [90]:
rdr=rdr[["participant_id","email", "phone_number",'participant_origin']]

In [30]:
rdr.to_csv("..\\Downloads\\email_phone.csv")

## first name, last name, and date of birth

In [91]:
######################################
print('Getting the data from the database...')
##########################431############

query = ('''  
    SELECT 
       ps.first_name,ps.last_name,ps.phone_number,ps.email,date_of_birth,ps.participant_id,o.display_name,ps.organization_id,ps.hpo_id,ps.participant_origin,ps.consent_for_study_enrollment_authored,
       p.is_ghost_id, ps.zip_code,ps.state_id,city,ps.street_address,ps.consent_for_electronic_health_records,ps.login_phone_number,num_baseline_samples_arrived ,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,
    CASE 
        WHEN
         (ps.hpo_id = 21 or p.is_ghost_id is not null) or
         (lower(ps.email) LIKE '%@example.com') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.login_phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%')
           THEN 'Yes'
           else "No" END as fake_test_participant_flag
    FROM participant_summary as ps
    left join organization as o on o.organization_id=ps.organization_id
    left outer join participant as p USING(participant_id)
    where is_test_participant = 0 and p.withdrawal_status = 1 and (ps.first_name, ps.last_name, ps.date_of_birth) in
                                                            (SELECT
                                                                ps.first_name, ps.last_name, ps.date_of_birth
                                                            FROM participant AS p
                                                            Left outer Join participant_summary as ps
                                                            on p.participant_id=ps.participant_id
                                                            Group By 1,2,3
                                                            having count(*)>1
                                                            )
                                                                    
    order by 1,2,3
''')

#Now execute the query and put it it into a dataframe
rdr = pd.read_sql(query, 
                     con=cnx)

print('Done!')

Getting the data from the database...
Done!


In [ ]:
######################################
print('Getting the data from the database...')
##########################431############

query = ('''  
    SELECT 
       ps.first_name,ps.last_name,ps.phone_number,ps.email,date_of_birth,ps.participant_id,o.display_name,ps.organization_id,ps.hpo_id,ps.participant_origin,ps.consent_for_study_enrollment_authored,
       p.is_ghost_id, ps.zip_code,ps.state_id,city,ps.street_address,ps.consent_for_electronic_health_records,ps.login_phone_number,num_baseline_samples_arrived ,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,
    CASE 
        WHEN
         (ps.hpo_id = 21 or p.is_ghost_id is not null) or
         (lower(ps.email) LIKE '%@example.com') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.login_phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%')
           THEN 'Yes'
           else "No" END as fake_test_participant_flag
    FROM participant_summary as ps
    left join organization as o on o.organization_id=ps.organization_id
    left outer join participant as p USING(participant_id)
    where (ps.first_name, ps.last_name, ps.date_of_birth) in
                                                            (SELECT
                                                                ps.first_name, ps.last_name, ps.date_of_birth
                                                            FROM participant AS p
                                                            Left outer Join participant_summary as ps
                                                            on p.participant_id=ps.participant_id
                                                            Group By 1,2,3
                                                            having count(*)>1
                                                            )
                                                                    
    order by 1,2,3
''')

#Now execute the query and put it it into a dataframe
rdr = pd.read_sql(query, 
                     con=cnx)

print('Done!')

Getting the data from the database...


In [33]:
rdr.shape

(48075, 23)

In [34]:
rdr['email']=rdr['email'].str.lower()

In [35]:
rdr['last_name']=rdr['last_name'].str.lower()

In [36]:
rdr['first_name']=rdr['first_name'].str.lower()

In [37]:
rdr['flag_1'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth','email','phone_number'],keep=False), 'Yes','No')

In [38]:
rdr['flag_2'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth'],keep=False), 'Yes','No')

In [39]:
rdr['flag_2_case_control'] = rdr['flag_2'] 

In [40]:
rdr['flag_3'] = np.where(rdr.duplicated(['email','phone_number'],keep=False), 'Yes','No')

In [41]:
rdr['flag_3_case_control'] = rdr['flag_3']

In [42]:
rdr.loc[rdr['flag_3_case_control'] == 'Yes', 'flag_2'] = 'No'

In [43]:
rdr.loc[rdr['flag_2_case_control'] == 'Yes', 'flag_3'] = 'No'

In [44]:
rdr.tail(10)

,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,hpo_id,participant_origin,...,num_baseline_samples_arrived,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,fake_test_participant_flag,flag_1,flag_2,flag_2_case_control,flag_3,flag_3_case_control
48065,zulekha,bawa,3237878754,None,1976-01-17,939801838,None,NaN,0,vibrent,...,0,1,1.0,2,Yes,No,Yes,Yes,No,No
48066,zulma,capestany,None,zulma.capestany1960@gmail.com,1960-01-03,481764287,None,NaN,5,vibrent,...,0,0,0.0,2,No,No,Yes,Yes,No,No
48067,zulma,capestany,8607764937,None,1960-01-03,396680955,"Community Health Center, Inc",12.0,5,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
48068,zulma,lopez,9492206115,zulmalopez23@icloud.com,1998-11-23,771975064,University of California Irvine,6.0,12,vibrent,...,0,0,0.0,1,No,No,Yes,Yes,No,No
48069,zulma,lopez,9492206115,None,1998-11-23,564677853,University of California Irvine,6.0,12,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
48070,zumara,labori,7863349489,None,1967-05-06,172239462,University of Miami,32.0,24,vibrent,...,0,0,0.0,1,No,No,Yes,Yes,No,No
48071,zumara,labori,7863349489,zumaralabori67@gmail.com,1967-05-06,840101343,University of Miami,32.0,24,vibrent,...,0,1,1.0,2,No,No,Yes,Yes,No,No
48072,zylinia,robinson,7732202813,zar1robinson@yahoo.com,1977-08-14,939724256,Rush University,19.0,3,vibrent,...,6,1,NaN,3,No,Yes,No,Yes,No,Yes
48073,zylinia,robinson,7732202813,zar1robinson@yahoo.com,1977-08-14,975101295,Northwestern University,16.0,3,vibrent,...,8,1,NaN,3,No,Yes,No,Yes,No,Yes
48074,zylinia,robinson,7732202813,None,1977-08-14,870697434,Rush University,19.0,3,vibrent,...,0,0,NaN,2,No,No,Yes,Yes,No,No


In [45]:
rdr['first_name'] = rdr['first_name'].str.strip()

In [46]:
rdr['last_name'] = rdr['last_name'].str.strip()

In [47]:
rdr['first_name'] = rdr['first_name'].str.replace('é','e')
rdr['first_name'] = rdr['first_name'].str.replace('í','i')
rdr['last_name'] = rdr['last_name'].str.replace('é','e')
rdr['last_name'] = rdr['last_name'].str.replace('í','i')
rdr['last_name'] = rdr['last_name'].str.replace('á','a')
rdr['last_name'] = rdr['last_name'].str.replace('ó','o')

In [48]:
vibrent = rdr.loc[rdr['participant_origin'] == 'vibrent'] 


In [49]:
vibrent

,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,hpo_id,participant_origin,...,num_baseline_samples_arrived,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,fake_test_participant_flag,flag_1,flag_2,flag_2_case_control,flag_3,flag_3_case_control
0,"'""><asd{{7*7}}","'""><asd{{7*7}}",1231515151,ahmed_hamada@wearehackerone.com,1993-02-03,823463681,None,NaN,21,vibrent,...,0,1,0.0,2,Yes,Yes,No,Yes,No,Yes
1,"'""><asd{{7*7}}","'""><asd{{7*7}}",1231515151,ahmed_hamada@wearehackerone.com,1993-02-03,705391345,None,NaN,21,vibrent,...,0,0,0.0,1,Yes,Yes,No,Yes,No,Yes
2,<b>foo,<b>foo,5417543010,kickino+b@wearehackerone.com,2000-01-01,663927729,None,NaN,21,vibrent,...,0,0,0.0,1,Yes,No,Yes,Yes,No,No
3,<b>foo,<b>foo,5417543011,kickino@wearehackerone.com,2000-01-01,594763398,None,NaN,21,vibrent,...,0,1,0.0,2,Yes,No,Yes,Yes,No,No
4,${{7*7}},"""><h1>asdasd</h1>",4342343256,yipman+yipman@wearehackerone.com,1992-04-16,998871219,None,NaN,21,vibrent,...,0,1,0.0,2,Yes,Yes,No,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48070,zumara,labori,7863349489,None,1967-05-06,172239462,University of Miami,32.0,24,vibrent,...,0,0,0.0,1,No,No,Yes,Yes,No,No
48071,zumara,labori,7863349489,zumaralabori67@gmail.com,1967-05-06,840101343,University of Miami,32.0,24,vibrent,...,0,1,1.0,2,No,No,Yes,Yes,No,No
48072,zylinia,robinson,7732202813,zar1robinson@yahoo.com,1977-08-14,939724256,Rush University,19.0,3,vibrent,...,6,1,NaN,3,No,Yes,No,Yes,No,Yes
48073,zylinia,robinson,7732202813,zar1robinson@yahoo.com,1977-08-14,975101295,Northwestern University,16.0,3,vibrent,...,8,1,NaN,3,No,Yes,No,Yes,No,Yes


In [50]:
###The Basics
rdr_bas = rdr.loc[rdr['questionnaire_on_the_basics'] == 1] 
vib_bas= vibrent.loc[vibrent['questionnaire_on_the_basics'] == 1] 

In [51]:
rdr_bas.count()

first_name                               38186
last_name                                38186
phone_number                             30659
email                                    28437
date_of_birth                            38186
participant_id                           38186
display_name                             25315
organization_id                          25315
hpo_id                                   38186
participant_origin                       38186
consent_for_study_enrollment_authored    38186
is_ghost_id                                299
zip_code                                 38178
state_id                                 38174
city                                     38174
street_address                           38174
consent_for_electronic_health_records    38186
login_phone_number                       16405
num_baseline_samples_arrived             38186
questionnaire_on_the_basics              38186
consent_for_genomics_ror                 29700
enrollment_st

In [52]:
vib_bas.count()

first_name                               30990
last_name                                30990
phone_number                             30659
email                                    22525
date_of_birth                            30990
participant_id                           30990
display_name                             24653
organization_id                          24653
hpo_id                                   30990
participant_origin                       30990
consent_for_study_enrollment_authored    30990
is_ghost_id                                299
zip_code                                 30982
state_id                                 30978
city                                     30978
street_address                           30978
consent_for_electronic_health_records    30990
login_phone_number                       13012
num_baseline_samples_arrived             30990
questionnaire_on_the_basics              30990
consent_for_genomics_ror                 22504
enrollment_st

In [53]:
###consent_for_genomics_ror
rdr_gror = rdr.loc[rdr['consent_for_genomics_ror'] == 1] 
vibrent.loc[vibrent['consent_for_genomics_ror'] == 1] 

,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,hpo_id,participant_origin,...,num_baseline_samples_arrived,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,fake_test_participant_flag,flag_1,flag_2,flag_2_case_control,flag_3,flag_3_case_control
50,aarin,austin,2055636920,aarinaustin.jones@gmail.com,1992-01-14,687559141,None,NaN,22,vibrent,...,0,1,1.0,2,No,No,Yes,Yes,No,No
51,aarin,austin,2055636920,aarinlashaye@yahoo.com,1992-01-14,774835550,University of Alabama Birmingham,33.0,22,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
52,aaron,alvarado,2082260265,aaron3az@gmail.com,1968-02-23,924119607,VA Phoenix Healthcare System,64.0,15,vibrent,...,1,1,1.0,3,No,No,Yes,Yes,No,No
53,aaron,alvarado,9282283620,aaron1az@yahoo.com,1968-02-23,263579273,None,NaN,15,vibrent,...,0,1,1.0,2,No,No,Yes,Yes,No,No
54,aaron,archibald,7734030184,None,1988-01-27,144766459,University of Illinois at Chicago,18.0,3,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48063,zoraida,diaz,2399610574,zoraidadiaz81@gmail.com,1969-11-24,194859026,"University of California, San Diego",4.0,12,vibrent,...,0,0,1.0,2,No,No,Yes,Yes,No,No
48065,zulekha,bawa,3237878754,None,1976-01-17,939801838,None,NaN,0,vibrent,...,0,1,1.0,2,Yes,No,Yes,Yes,No,No
48067,zulma,capestany,8607764937,None,1960-01-03,396680955,"Community Health Center, Inc",12.0,5,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
48069,zulma,lopez,9492206115,None,1998-11-23,564677853,University of California Irvine,6.0,12,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No


In [54]:
rdr_gror

,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,hpo_id,participant_origin,...,num_baseline_samples_arrived,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,fake_test_participant_flag,flag_1,flag_2,flag_2_case_control,flag_3,flag_3_case_control
18,a,b,None,nino+ao84@owaves.com,1995-04-01,239827063,None,NaN,21,careevolution,...,0,1,1.0,2,Yes,No,Yes,Yes,No,No
19,a,b,None,nino@owaves.com,1995-04-01,668657296,None,NaN,0,careevolution,...,0,1,1.0,2,No,No,Yes,Yes,No,No
30,a,b,None,nino+ao114@owaves.com,1995-04-01,568976115,None,NaN,21,careevolution,...,0,0,1.0,2,Yes,No,Yes,Yes,No,No
33,a,b,None,nino+ao72@owaves.com,1995-04-01,408518861,None,NaN,21,careevolution,...,0,1,1.0,2,Yes,No,Yes,Yes,No,No
34,a,b,None,nino+ao120@owaves.com,1995-04-01,833903441,None,NaN,21,careevolution,...,0,1,1.0,2,Yes,No,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48063,zoraida,diaz,2399610574,zoraidadiaz81@gmail.com,1969-11-24,194859026,"University of California, San Diego",4.0,12,vibrent,...,0,0,1.0,2,No,No,Yes,Yes,No,No
48065,zulekha,bawa,3237878754,None,1976-01-17,939801838,None,NaN,0,vibrent,...,0,1,1.0,2,Yes,No,Yes,Yes,No,No
48067,zulma,capestany,8607764937,None,1960-01-03,396680955,"Community Health Center, Inc",12.0,5,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
48069,zulma,lopez,9492206115,None,1998-11-23,564677853,University of California Irvine,6.0,12,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No


In [56]:
###Ehr status
rdr.loc[rdr['ehr_status'] == 1] 
vibrent.loc[vibrent['ehr_status'] == 1] 

KeyError: 'ehr_status'

In [57]:
###The biosample
rdr_bio = rdr.loc[rdr['num_baseline_samples_arrived'] > 0] 
vib_bio= vibrent.loc[vibrent['num_baseline_samples_arrived'] > 0] 

In [58]:
vib_bio.count()

first_name                               15096
last_name                                15096
phone_number                             14936
email                                     9214
date_of_birth                            15096
participant_id                           15096
display_name                             15081
organization_id                          15081
hpo_id                                   15096
participant_origin                       15096
consent_for_study_enrollment_authored    15096
is_ghost_id                                 11
zip_code                                 15092
state_id                                 15092
city                                     15092
street_address                           15092
consent_for_electronic_health_records    15096
login_phone_number                        8729
num_baseline_samples_arrived             15096
questionnaire_on_the_basics              15096
consent_for_genomics_ror                  9756
enrollment_st

In [59]:
rdr.loc[rdr['enrollment_status'] == 3] 

,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,hpo_id,participant_origin,...,num_baseline_samples_arrived,questionnaire_on_the_basics,consent_for_genomics_ror,enrollment_status,fake_test_participant_flag,flag_1,flag_2,flag_2_case_control,flag_3,flag_3_case_control
51,aarin,austin,2055636920,aarinlashaye@yahoo.com,1992-01-14,774835550,University of Alabama Birmingham,33.0,22,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
52,aaron,alvarado,2082260265,aaron3az@gmail.com,1968-02-23,924119607,VA Phoenix Healthcare System,64.0,15,vibrent,...,1,1,1.0,3,No,No,Yes,Yes,No,No
54,aaron,archibald,7734030184,None,1988-01-27,144766459,University of Illinois at Chicago,18.0,3,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
57,aaron,buck,7143074511,None,1982-10-22,767070644,University of California Irvine,6.0,12,vibrent,...,7,1,1.0,3,No,No,Yes,Yes,No,No
59,aaron,cherry,2565038896,None,1978-07-03,989621723,UAB Medicine Huntsville,72.0,22,vibrent,...,8,1,NaN,3,No,No,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48064,zulekha,bawa,3106428851,None,1976-01-17,235061242,University of Southern California,5.0,12,vibrent,...,3,1,NaN,3,No,No,Yes,Yes,No,No
48067,zulma,capestany,8607764937,None,1960-01-03,396680955,"Community Health Center, Inc",12.0,5,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
48069,zulma,lopez,9492206115,None,1998-11-23,564677853,University of California Irvine,6.0,12,vibrent,...,8,1,1.0,3,No,No,Yes,Yes,No,No
48072,zylinia,robinson,7732202813,zar1robinson@yahoo.com,1977-08-14,939724256,Rush University,19.0,3,vibrent,...,6,1,NaN,3,No,Yes,No,Yes,No,Yes


In [60]:
rdr_bas['flag_1'] = np.where(rdr_bas.duplicated(['first_name', 'last_name','date_of_birth','email','phone_number'],keep=False), 'Yes','No')

In [62]:
rdr_bas['flag_2'] = np.where(rdr_bas.duplicated(['first_name', 'last_name','date_of_birth'],keep=False), 'Yes','No')

In [63]:
rdr_bas['flag_3'] = np.where(rdr_bas.duplicated(['email','phone_number'],keep=False), 'Yes','No')

In [64]:
rdr_bas.to_csv("..\\Downloads\\testing.csv")

In [65]:
rdr.to_csv("..\\Downloads\\first_last.csv")

In [66]:
duplicated_first_last_date_of_birth=rdr[["first_name","last_name","phone_number","email","ehr_status","participant_origin","flag_1","flag_2"]]



KeyError: "['ehr_status'] not in index"

In [67]:
df_2 = duplicated_first_last_date_of_birth.groupby(["flag_1",'flag_2'])["ehr_status"].value_counts()
df_2.drop_duplicates()

NameError: name 'duplicated_first_last_date_of_birth' is not defined

In [57]:
import copy
deep_copy = copy.deepcopy(duplicated_first_last_date_of_birth)


In [58]:
result1 = pd.merge(df, duplicated_first_last_date_of_birth, on='flag_1', how='inner')

In [59]:
result2 = pd.merge(df_2, deep_copy, on='flag_2', how='inner')

In [60]:
final = pd.concat([result1,result2])

In [61]:
final =final.drop_duplicates()

In [62]:
final.to_csv("..\\Downloads\\final.csv")

In [63]:
final

,flag_1,withdrawal_status_x,first_name,last_name,phone_number,email,withdrawal_status_y,participant_origin,flag_2
0,No,35847,<b>foo,<b>foo,5417543010,kickino+b@wearehackerone.com,1,vibrent,Yes
1,No,35847,<b>foo,<b>foo,5417543011,kickino@wearehackerone.com,1,vibrent,Yes
2,No,35847,659,arizona,1234567890,tqaio@example.com,1,vibrent,Yes
3,No,35847,659,arizona,2345678901,659@example.com,1,vibrent,Yes
4,No,35847,a,a,1111111111,appscan2@vibrenthealth.com,2,vibrent,Yes
...,...,...,...,...,...,...,...,...,...
90713,No,2861,zulma,capestany,None,zulma.capestany1960@gmail.com,1,vibrent,Yes
90714,No,2861,zulma,capestany,8607764937,None,1,vibrent,Yes
90715,No,2861,zulma,lopez,9492206115,zulmalopez23@icloud.com,1,vibrent,Yes
90716,No,2861,zulma,lopez,9492206115,None,1,vibrent,Yes


### How many accounts are there per count

In [46]:
rdr=rdr[['participant_id','first_name','last_name','date_of_birth','participant_origin','withdrawal_status']]

KeyError: "['withdrawal_status'] not in index"

In [65]:
rdr

,participant_id,first_name,last_name,date_of_birth,participant_origin,withdrawal_status
0,823463681,"'""><asd{{7*7}}","'""><asd{{7*7}}",1993-02-03,vibrent,2
1,705391345,"'""><asd{{7*7}}","'""><asd{{7*7}}",1993-02-03,vibrent,1
2,663927729,<b>foo,<b>foo,2000-01-01,vibrent,1
3,594763398,<b>foo,<b>foo,2000-01-01,vibrent,1
4,219016202,${{7*7}},"""><h1>asdasd</h1>",1992-04-16,vibrent,1
...,...,...,...,...,...,...
40908,771975064,zulma,lopez,1998-11-23,vibrent,1
40909,564677853,zulma,lopez,1998-11-23,vibrent,1
40910,975101295,zylinia,robinson,1977-08-14,vibrent,2
40911,870697434,zylinia,robinson,1977-08-14,vibrent,1


In [66]:
rdr.to_csv("..\\Downloads\\name_DOB.csv")

##  (First, Last DOB) & (Email & Phone)

In [47]:
######################################
print('Getting the data from the database...')
##########################431############

query = ('''  
    SELECT
       ps.participant_id,ps.participant_origin,ps.first_name,ps.last_name,ps.phone_number,ps.email,date_of_birth,ps.participant_id,o.display_name,ps.organization_id,ps.hpo_id,ps.participant_origin,ps.consent_for_study_enrollment_authored,
       p.is_ghost_id, ps.zip_code,ps.state_id,city,ps.street_address,ps.ehr_status,ps.login_phone_number, 
    CASE 
        WHEN
         (ps.hpo_id = 21 or p.is_ghost_id is not null) or
         (lower(ps.email) LIKE '%@example.com') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%') or
         (REPLACE(REPLACE(REPLACE(REPLACE(ps.login_phone_number, '(', ''), ')', ''), '-', ''), ' ', '') like '4442%')
           THEN 'Yes'
           else "No" END as fake_test_participant_flag
    FROM participant_summary as ps
    left join organization as o on o.organization_id=ps.organization_id
    left outer join participant as p USING(participant_id)
    where 
        (phone_number,email,ps.first_name, ps.last_name, ps.date_of_birth) in
                                                        (SELECT
                                                            phone_number,email,ps.first_name, ps.last_name, ps.date_of_birth
                                                        FROM participant AS p
                                                        Left outer Join participant_summary as ps
                                                        on p.participant_id=ps.participant_id
                                                        Group By 1,2,3,4,5
                                                        having count(*)>1
                                                        )
                                                         
    order by 1,2,3
''')

#Now execute the query and put it it into a dataframe
rdr = pd.read_sql(query, 
                     con=cnx)

print('Done!')

Getting the data from the database...
Done!


In [48]:
rdr.shape

(1792, 21)

In [49]:
rdr['email']=rdr['email'].str.lower()

In [50]:
rdr['last_name']=rdr['last_name'].str.lower()

In [51]:
rdr['first_name']=rdr['first_name'].str.lower()

In [52]:
rdr['phone_number']=rdr['phone_number'].str.lower()

In [53]:
rdr['flag_1'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth','email','phone_number'],keep=False), 'Yes','No')

In [54]:
rdr['flag_2'] = np.where(rdr.duplicated(['first_name', 'last_name','date_of_birth'],keep=False), 'Yes','No')

In [55]:
rdr['flag_3'] = np.where(rdr.duplicated(['email','phone_number'],keep=False), 'Yes','No')

In [56]:
rdr.loc[rdr['flag_3'] == 'Yes', 'flag_2'] = 'No'

In [57]:
rdr.loc[rdr['flag_2'] == 'Yes', 'flag_3'] = 'No'

In [58]:
rdr.tail(10)

,participant_id,participant_origin,first_name,last_name,phone_number,email,date_of_birth,participant_id,display_name,organization_id,...,zip_code,state_id,city,street_address,ehr_status,login_phone_number,fake_test_participant_flag,flag_1,flag_2,flag_3
1782,994630566,vibrent,alec,watson,2074054261,aleccole32@gmail.com,1981-01-05,994630566,Mass General Brigham,22.0,...,04101,660.0,Portland,94 Park Avenue,0,None,No,Yes,No,Yes
1783,994882803,vibrent,michael,plummer,2565201572,michaelplummer61@gmail.com,1955-03-04,994882803,None,NaN,...,35810,682.0,Huntsville,2205 Evans av NW,0,None,Yes,Yes,No,Yes
1784,994916899,vibrent,dominique,mcclain,7089688207,dmcclain8761@gmail.com,1987-07-21,994916899,University of Illinois at Chicago,18.0,...,60649,683.0,Chicago,7251 s South shore drive,1,None,No,Yes,No,Yes
1785,994926831,vibrent,shawn,tillie,5042949012,shawntillie17@gmail.com,1973-05-20,994926831,None,NaN,...,70125,654.0,New Orleans,4500 claiborne Ave,0,None,No,Yes,No,Yes
1786,995093683,vibrent,diana,peterson,6084401770,dmpeterson321@gmail.com,1965-01-23,995093683,"University of Wisconsin, Madison",52.0,...,53711,667.0,Madison,708 Odana Lane,1,None,No,Yes,No,Yes
1787,996130090,vibrent,jean,pointon,2053104885,dr_jean@bellsouth.net,1960-01-12,996130090,None,NaN,...,35401,682.0,Tuscaloosa,1220 Queen City Avenue,0,None,No,Yes,No,Yes
1788,998253233,vibrent,alfreda,bonner,2054201914,alfredab50@hotmail.com,1956-04-26,998253233,None,NaN,...,35221,682.0,Birmingham,4016 Gray Avenue S.W.,0,None,No,Yes,No,Yes
1789,998673845,vibrent,nicole,johnson,9206664568,toyd34@yahoo.com,1979-01-08,998673845,"University of Wisconsin, Madison",52.0,...,54956,667.0,Neenah,845 e cecil st,0,None,No,Yes,No,Yes
1790,998756412,vibrent,rajendra,shah,2482317162,rajendrashah701@gmail.com,1944-12-11,998756412,Henry Ford Health System,44.0,...,48331,643.0,Farmington Hills,35338 Glengary Circle,1,2482317162,No,Yes,No,Yes
1791,998871219,vibrent,${{7*7}},"""><h1>asdasd</h1>",4342343256,yipman+yipman@wearehackerone.com,1992-04-16,998871219,None,NaN,...,10001,685.0,awedasd,"""><h1>asdasd</h1>",0,None,Yes,Yes,No,Yes


In [59]:
rdr['last_name'] = rdr['last_name'].str.replace('á','a')
rdr['first_name'] = rdr['first_name'].str.replace('é','e')

In [60]:
rdr['participant_origin'].count()

participant_origin    1792
participant_origin    1792
dtype: int64

In [61]:
rdr.to_csv("..\\Downloads\\all.csv")

### How many accounts are there per count

In [85]:
rdr=rdr[['participant_id','first_name', 'last_name','date_of_birth','email','phone_number','participant_origin','withdrawal_status']]

In [86]:
rdr['first_name'] = rdr['first_name'].str.strip()

In [87]:
rdr['last_name'] = rdr['last_name'].str.strip()

In [88]:
rdr.to_csv("..\\Downloads\\all.csv")